In [3]:
from chessdotcom import client
import pandas as pd

In [192]:
user_name = "kirkey02"

In [209]:
response = client.get_player_profile(user_name)

response.player

namespace(avatar='https://images.chesscomfiles.com/uploads/v1/user/39309608.2e314836.200x200o.c114f3211bea.jpeg',
          player_id=39309608,
          id='https://api.chess.com/pub/player/kirkey02',
          url='https://www.chess.com/member/kirkey02',
          username='kirkey02',
          followers=4,
          country='https://api.chess.com/pub/country/GB',
          last_online=1668095006,
          joined=1507569159,
          status='premium',
          is_streamer=False,
          verified=False)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [194]:
games_list = []

for month in range(1,12):
    resp = client.get_player_games_by_month_pgn(user_name, 2022, month)
    games_response_text = resp.pgn.pgn
    games_response_list = games_response_text.split("\n")


    games_chunks = chunks(games_response_list, 25)


    for game_list in games_chunks:
        game_dict = {}
        for line in game_list:
            if len(line) > 0:
                if line[0] == '1':
                    game_dict['Moves'] = line
                else:
                    key = line.split(" ")[0][1:]
                    value = line.split('"')[1]
                    game_dict[key] = value

        games_list.append(game_dict)

In [195]:
chess_df = pd.DataFrame(games_list).sort_values(['Date'])

In [196]:
chess_df['W/L_Points'] = chess_df.apply(lambda row: 1 if row['Result'] == "1-0" else -1 if row['Result'] == "0-1" else 0, axis=1)
chess_df['ResultWords'] = chess_df.apply(lambda row: "Win" if row['Result'] == "1-0" else "Loss" if row['Result'] == "0-1" else "Draw", axis=1)

In [197]:
chess_df.groupby(["ECO"]).sum().sort_values('W/L_Points', ascending=False)

,W/L_Points
ECO,
D02,20
D00,18
D06,18
A04,13
A41,7
...,...
B11,-7
C55,-7
A43,-10


In [198]:
chess_df['Colour'] = chess_df.apply(lambda row: "White" if user_name in row['White'] else "Black", axis=1)

In [202]:
def check_victory_method(termination):
    if "checkmate" in termination:
        return "Checkmate"
    if "resignation" in termination:
        return "Resignation"
    if "time" in termination:
        return "Time"
    if "stalemate" in termination:
        return "Stalemate"
    if "abandoned" in termination:
        return "Abandoned"
    if "repetition" in termination:
        return "Repetition"
    if "insufficient material" in termination:
        return "Insufficient material"
    if "Game drawn by agreement" in termination:
        return "Game drawn by agreement"
    else:
        return 0

In [203]:
chess_df["VictoryMethod"] = chess_df.apply(lambda row: check_victory_method(row['Termination']), axis=1)

assert len(chess_df[chess_df["VictoryMethod"]==0]) == 0

In [205]:
def check_time_mode(time_control):
    if "/" in time_control:
        return "Daily"
    time_control = int(time_control.split("+")[0])
    if time_control < 180:
        return "Bullet"
    if time_control >= 600:
        return "Rapid"
    if time_control < 600:
        return "Blitz"
chess_df["TimeMode"] = chess_df.apply(lambda row: check_time_mode(row['TimeControl']), axis=1)

In [206]:
chess_df['Rating'] = chess_df.apply(lambda row: row['WhiteElo'] if row['Colour'] == 'White' else row['BlackElo'], axis=1)
chess_df['OpponentRating'] = chess_df.apply(lambda row: row['BlackElo'] if row['Colour'] == 'White' else row['WhiteElo'], axis=1)

chess_df.iloc[100]

Event                                                     Live Chess
Site                                                       Chess.com
Date                                                      2022.01.17
Round                                                              -
White                                                      Matiax317
Black                                                       kirkey02
Result                                                           0-1
CurrentPosition     1r4k1/5ppp/Q3p3/2pp4/8/3bP3/P2N1PPP/2q1K2R w K -
Timezone                                                         UTC
ECO                                                              D00
ECOUrl             https://www.chess.com/openings/Queens-Pawn-Ope...
UTCDate                                                   2022.01.17
UTCTime                                                     22:44:24
WhiteElo                                                        1317
BlackElo                          

In [207]:
chess_df.to_csv("chess_stats.csv")